In [1]:
from datetime import datetime, date, timedelta

# Get today's date
today = datetime.now()

# Number of days from today
n_3 = 3
n_2 = 2
n_1 = 1

# Calculate yesterday's date
yesterday = today - timedelta(days = n_1)

# Account for weekends
if yesterday.strftime("%A") == "Sunday":
  yesterday = today - timedelta(days = n_3)
  
elif yesterday.strftime("%A") == "Saturday":
  yesterday = today - timedelta(days = n_2)

else:
  yesterday

# Print yesterday's date
print("Yesterday's date:", yesterday.date())

Yesterday's date: 2024-11-15


# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
!pip install xlsxwriter

In [3]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import time

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [5]:
from secrets_py import MARKETSTACK_API_TOKEN
from secrets_py import TIINGO_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
!pip install bs4

In [7]:
import requests
from bs4 import BeautifulSoup
import re
import json

## Select stocks from the Dow 30

# Step 1: Get the webpage content
url = 'https://www.cnbc.com/dow-30/'  # Example CNBC page
response = requests.get(url)
webpage_content = response.text

# Step 2: Parse HTML using BeautifulSoup
soup = BeautifulSoup(webpage_content, 'html.parser')

# Step 3: Use regex to find the JavaScript object
pattern = r'window\.__s_data\s*=\s*(\{.*?\});'
script_tag = soup.find("script", text=re.compile(pattern))

if script_tag:
    # Step 4: Extract the JavaScript object
    match = re.search(pattern, script_tag.string)
    if match:
        js_object = match.group(1)
        
        # Step 5: Convert JavaScript object to Python dictionary
        js_object = js_object.replace("null", "None").replace("false", "False").replace("true", "True")
        data = eval(js_object)
        
        # Step 6: Navigate to the ticker symbols within the data structure
        ticker_symbols = []
        try:
            # Digging into the 'marketsSectionTable' part to find ticker symbols
            market_table = data['page']['page']['layout']
            for layout in market_table:
                for column in layout['columns']:
                    for module in column['modules']:
                        if module['name'] == 'marketsSectionTable':
                            for chart in module['data']['relatedMarketCharts']:
                                for issue in chart['chartIssueDetail']:
                                    ticker_symbols.append(issue['symbol'])
        
            # Step 7: Print the extracted ticker symbols
            print("Extracted Ticker Symbols:")
            print(ticker_symbols)
        except KeyError as e:
            print(f"Key not found: {e}")
    else:
        print("JavaScript object not found in script.")
else:
    print("Script tag not found.")

Extracted Ticker Symbols:
['AMZN', 'AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'SHW', 'TRV', 'UNH', 'CRM', 'NVDA', 'VZ', 'V', 'WMT', 'DIS']


C:\Users\Brian GT\AppData\Local\Temp\ipykernel_4732\2415861260.py:18: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script_tag = soup.find("script", text=re.compile(pattern))


In [8]:
# Adding a column based on a condition
stocks['is_dow_30'] = stocks['Ticker'].apply(lambda x: 1 if x in ticker_symbols else 0 )

# Selecting only the stocks in Dow 30 because only the info from the MarketStacks
select_stocks = stocks[stocks["is_dow_30"] == 1]

# Split the resulting table into sub-lists
# def chunks(lst, n):
#   """Yield successive n-sized chunks from lst."""
#   for i in range(0, len(lst), n):
#     yield lst[i:1+n]

select_stocks_list = select_stocks.to_dict("records")
select_stocks_list

[{'Ticker': 'AAPL', 'is_dow_30': 1},
 {'Ticker': 'AMGN', 'is_dow_30': 1},
 {'Ticker': 'AMZN', 'is_dow_30': 1},
 {'Ticker': 'AXP', 'is_dow_30': 1},
 {'Ticker': 'BA', 'is_dow_30': 1},
 {'Ticker': 'CAT', 'is_dow_30': 1},
 {'Ticker': 'CRM', 'is_dow_30': 1},
 {'Ticker': 'CSCO', 'is_dow_30': 1},
 {'Ticker': 'CVX', 'is_dow_30': 1},
 {'Ticker': 'DIS', 'is_dow_30': 1},
 {'Ticker': 'GS', 'is_dow_30': 1},
 {'Ticker': 'HD', 'is_dow_30': 1},
 {'Ticker': 'HON', 'is_dow_30': 1},
 {'Ticker': 'IBM', 'is_dow_30': 1},
 {'Ticker': 'JNJ', 'is_dow_30': 1},
 {'Ticker': 'JPM', 'is_dow_30': 1},
 {'Ticker': 'KO', 'is_dow_30': 1},
 {'Ticker': 'MCD', 'is_dow_30': 1},
 {'Ticker': 'MMM', 'is_dow_30': 1},
 {'Ticker': 'MRK', 'is_dow_30': 1},
 {'Ticker': 'MSFT', 'is_dow_30': 1},
 {'Ticker': 'NKE', 'is_dow_30': 1},
 {'Ticker': 'NVDA', 'is_dow_30': 1},
 {'Ticker': 'PG', 'is_dow_30': 1},
 {'Ticker': 'SHW', 'is_dow_30': 1},
 {'Ticker': 'TRV', 'is_dow_30': 1},
 {'Ticker': 'UNH', 'is_dow_30': 1},
 {'Ticker': 'V', 'is_dow_30

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [9]:
# Define your columns
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]

# Create an empty DataFrame with specified columns
final_dataframe = pd.DataFrame(columns=my_columns)

other_column = "N/A"

print(final_dataframe)

Empty DataFrame
Columns: [Ticker, Stock Price, Market Capitalization, Number of Shares to Buy]
Index: []


### 1. Get

In [10]:
headers = {
    'Content-Type': 'application/json'
}

final_dataframe = pd.DataFrame(columns=my_columns)

for stock in select_stocks["Ticker"][:30]:
  # To get the closing stock price as of the previous day
  price_url = f"https://api.tiingo.com/tiingo/daily/{stock}/prices?token={TIINGO_API_TOKEN}&startDate={yesterday}"
  response_1 = requests.get(price_url, headers=headers)
  if response_1.status_code == 200:
    price_list = response_1.json()
    price_dict = price_list[0]
    latest_price = price_dict['close']
    new_data = pd.DataFrame([price_dict]) # Create a DataFrame from the API data. Here, we are assuming that tk_data is a dictionary of key-value pairs
    row_index = final_dataframe[final_dataframe['Ticker'] == stock].index # Find the index of the row in select_stocks where 'Ticker' matches
    if not row_index.empty: # If a matching row is found, update the row with the new data
      row_index = row_index[0]  # Get the first match index
      final_dataframe.update(price_dict, overwrite=True)
    else:
      print(f"No matching ticker found for {stock}")
  elif response_1.status_code == 429:
    # retry_after = int(response_1.headers.get('Retry-After', 60))  # Default to 60 seconds if not specified
    print(f"Rate limit hit for {stock}: {response_1.status_code}")
    # print(f"Rate limit hit. Retrying after {retry_after} seconds...")
    # time.sleep(retry_after)  # Wait for the specified time before retrying
  else:
    print(f"Failed to retrieve data for {stock}: {response_1.status_code}")

final_dataframe


No matching ticker found for AAPL
No matching ticker found for AMGN
No matching ticker found for AMZN
No matching ticker found for AXP
No matching ticker found for BA
No matching ticker found for CAT
No matching ticker found for CRM
No matching ticker found for CSCO
No matching ticker found for CVX
No matching ticker found for DIS
No matching ticker found for GS
No matching ticker found for HD
No matching ticker found for HON
No matching ticker found for IBM
No matching ticker found for JNJ
No matching ticker found for JPM
No matching ticker found for KO
No matching ticker found for MCD
No matching ticker found for MMM
No matching ticker found for MRK
No matching ticker found for MSFT
No matching ticker found for NKE
No matching ticker found for NVDA
No matching ticker found for PG
No matching ticker found for SHW
No matching ticker found for TRV
No matching ticker found for UNH
No matching ticker found for V
No matching ticker found for VZ
No matching ticker found for WMT


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [11]:
headers = {
    'Content-Type': 'application/json'
}

final_dataframe = pd.DataFrame(columns=my_columns)

for stock in select_stocks["Ticker"][:30]:
  # To get the market capitalization value as of the previous day
  market_cap_url = f"https://api.tiingo.com/tiingo/fundamentals/{stock}/daily?token={TIINGO_API_TOKEN}&startDate={yesterday}"
  response_2 = requests.get(market_cap_url, headers=headers)
  if response_2.status_code == 200:
    marcap_list = response_2.json()
    marcap_dict = marcap_list[0]
    market_cap = marcap_dict['marketCap']
    new_data = pd.DataFrame([marcap_dict]) # Create a DataFrame from the API data. Here, we are assuming that tk_data is a dictionary of key-value pairs
    row_index = final_dataframe[final_dataframe['Ticker'] == stock].index   # Find the index of the row in select_stocks where 'Ticker' matches
    if not row_index.empty: # If a matching row is found, update the row with the new data
      row_index = row_index[0]  # Get the first match index
      final_dataframe.update(marcap_dict, overwrite=True)
    else:
      print(f"No matching ticker found for {stock}")
  elif response_1.status_code == 429:
    # retry_after = int(response_1.headers.get('Retry-After', 60))  # Default to 60 seconds if not specified
    print(f"Rate limit hit for {stock}: {response_2.status_code}")
    # print(f"Rate limit hit. Retrying after {retry_after} seconds...")
    # time.sleep(retry_after)  # Wait for the specified time before retrying
  else:
    print(f"Failed to retrieve data for {stock}: {response_2.status_code}")
  
final_dataframe

No matching ticker found for AAPL
No matching ticker found for AMGN
Failed to retrieve data for AMZN: 400
No matching ticker found for AXP
No matching ticker found for BA
No matching ticker found for CAT
No matching ticker found for CRM
No matching ticker found for CSCO
No matching ticker found for CVX
No matching ticker found for DIS
No matching ticker found for GS
No matching ticker found for HD
No matching ticker found for HON
No matching ticker found for IBM
No matching ticker found for JNJ
No matching ticker found for JPM
No matching ticker found for KO
No matching ticker found for MCD
No matching ticker found for MMM
No matching ticker found for MRK
No matching ticker found for MSFT
No matching ticker found for NKE
Failed to retrieve data for NVDA: 400
No matching ticker found for PG
Failed to retrieve data for SHW: 400
No matching ticker found for TRV
No matching ticker found for UNH
No matching ticker found for V
No matching ticker found for VZ
No matching ticker found for WMT


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [12]:
# Assuming an equal investment

for i in range(0, len(final_dataframe.index)):
  print(i)
  # final_dataframe["Number of Shares to Buy"]
  # position_size = val/len(final_dataframe.index)
  # number_of_shares = position_size/500
  # print(math.floor(number_of_shares))

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: